## Membros

- Amanda Oliveira
- Daniel Henrique
- Samuel Pedro

## Resumo

### Parâmetros

Será usado um kernel `rbf` em um kernel SVC. O `C` será otimizado pelo Optuna a fim de obter um resultado melhor. O `gamma` foi deixado no valor padrão que é `scale`, a fórmula referente a esse valor está disponível na documentação do scikit-learn.

### Representação adotada

A representação adotada foi escalar os valores númericos com um scaler do sklearn (no momento, um RobustScaler). Criar uma bag of words a partir do resumo. Criar uma bag of items com o elenco. Extrair o ano da data de estreia. E, por fim, descartar a história original e o titulo por não serem tão relevantes para os resultados. A coluna "adulo" foi deixada mesmo estando distribuida de maneira extremamente desuniforme, assumindo-se que o modelo irá incorporá-la melhor caso ela seja distribuída mais uniformemente.

Porém, o resultado mais efeitvo usa apenas a bag of words e a bag of items, descartando os demais fatores. Isso provavelmente está relacionado com a grande ausência de informações (NaNs) dos demais parâmetros.

### Método adotado

O método adotado foi um classificador SVC com um kernel `rbf`, já que sua acurácia se mostrou maior que o `linear`. O kernel `poly` não foi testado por falta de tempo, mas espera-se que ele seja menos preciso em grais baixos ou consideravelmente mais lento em grais altos. 

Os resultados do método Random Forest são levemente inferiores, mas conseguem incorporar mais informações para a predição.

## Pós-análise

Após algumas reflexões, percebeu-se que o modelo era altamente dependente ao resumo, o que não é algo necessariamente ruim. Por conta disso, decidimos adotar uma representação que consiste basicamente do resumo com uma limpeza aplicada por cima. Para os filmes que não se têm um resumo, usaremos um segundo modelo baseado em RandomForest para prever a categoria baseando-se nos demais parâmetros.

In [1]:
import pandas as pd
import numpy as np
from competicao_am.gerar_resultado_teste import gerar_saida_teste

In [2]:
df_treino = pd.read_csv('datasets/movies_amostra.csv')
df_predict_ex = pd.read_csv('datasets/movies_amostra_teste_ex.csv')
df_predict_final = pd.read_csv('datasets/movies_amostra_teste_1.csv')

df_treino.head(2)

,id,titulo,adulto,orcamento,idioma_original,popularidade,data_de_estreia,resumo,receita,duracao,genero,ator_1,ator_2,ator_3,ator_4,ator_5,dirigido_por,escrito_por_1,escrito_por_2,historia_original
0,86295,Treasure of the Yankee Zephyr,False,0,en,2.0,1981-11-28,In a lake high in the mountains of New Zealand...,0.0,108.0,Action,Ken Wahl,Lesley Ann Warren,Donald Pleasence,George Peppard,Bruno Lawrence,David Hemmings,Everett De Roche,NaN,NaN
1,289198,Redeemer,False,0,es,2.0,2014-09-18,A former hit-man for a drug cartel becomes a v...,0.0,88.0,Action,Marko Zaror,Loreto Aravena,Mauricio Diocares,José Luís Mósca,Noah Segan,Ernesto Díaz Espinoza,NaN,NaN,NaN


In [3]:
from competicao_am.preprocessamento_atributos_competicao import DataframePreprocessing
preprocessor = DataframePreprocessing(df_treino, df_treino, 'genero')

## Otimização dos parâmetros

In [4]:
import pickle
import json

import optuna
import matplotlib.pyplot as plt
import sklearn
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

from base_am.resultado import Fold
from base_am.avaliacao import Experimento
import competicao_am.metodo_competicao as mc
import competicao_am.avaliacao_competicao as ac

## Um pouco sobre SVC's

### Kernel

O `kernel` indica o tipo de "hiper-plano" usado para separar as classes.

Os kernels a serem considerados são: `linear`, `rbf` e `poly`

#### `poly`: degree

Grau do polinômio a ser usado. Quanto maior o valor, maior o tempo gasto para o treino.

### gamma

Parâmetro usado para "hiper-planos" não lineares. Quanto maior, mais ele tenta en encaixar o modelo perfeitamente.

Valores muito altos caminhão em direção de um _overfitting_.

### C

C é o parâmetro de penalidade do erro. Ele controla o prejuízo entre paredes de decisões suaves e classificar os pontos corretamente.



Links úteis:

- [A guide to SVM parameter tuning](https://towardsdatascience.com/a-guide-to-svm-parameter-tuning-8bfe6b8a452c)

In [5]:
def experimento(metodo: mc.MetodoCompeticaoValidacaoSVM, num_trials: int = 2, otimizador:ac.OtimizacaoObjetivo = ac.OtimizacaoObjetivoSVCRbfComGamma, ml_method = None, **kwargs) -> Experimento:
    folds = Fold.gerar_k_folds(df_treino, val_k=5, col_classe="genero",
                                num_repeticoes=1, num_folds_validacao=4, num_repeticoes_validacao=1)

    ClasseObjetivo = ac.WrapperOtimizacaoObjetivo(otimizador, metodo=metodo, **kwargs)

    experimento = Experimento(folds, ml_method=ml_method, ClasseObjetivoOtimizacao=ClasseObjetivo, num_trials=num_trials)

    return experimento

def best_parameters(experiment: Experimento):
    """
    Extração de parâmetros dos resultados
    """
    results = []
    for study in experiment.studies_per_fold:
        results.append((study.best_trial.value, study.best_trial.params))
    results.sort()
    return list(reversed(sorted(results)))

In [6]:
import pickle
import os

g_version = 2

def do_experiment(name: str, metodo: mc.MetodoCompeticaoValidacaoSVM = None, num_trials: int = None, otimizador:ac.OtimizacaoObjetivo = None, ml_method = None, force: bool = False, version: int = g_version, **kwargs) -> Experimento:
    """
    Função que executa os experimentos caso ainda não existam, caso já existam, os recupera
    """
    file_name = f'{name}__{version}.p'
    file_path = f'resultados/{file_name}'

    if not force and os.path.isfile(file_path):
        exp = None

        with open(file_path, 'rb') as exp_p:
            exp = pickle.load(exp_p)
            # Check instance here

        return exp
    
    if ml_method is None and (metodo is None or num_trials is None or otimizador is None):
        raise ValueError('To run the experiment, the appropriate parameters should be give (ml_method or metodo, num_trials and otimizador)')

    exp = experimento(metodo=metodo, num_trials=num_trials, otimizador=otimizador, ml_method=ml_method, **kwargs)
    macro_f1 = exp.macro_f1_avg

    with open(file_path, 'wb') as exp_p:
        # exp = pickle.dump(exp, exp_p)
        pass

    return exp

In [7]:
exp_test_rf = do_experiment('exp_test_rf', mc.MetodoCompeticaoValidacaoRF, 100, ac.OtimizacaoObjetivoRandomForest, version=2)

In [8]:
exp_test_svm = do_experiment('exp_test_svm', mc.MetodoCompeticaoValidacaoSVM, 100, ac.OtimizacaoObjetivoSVCRbfComGamma, version=2)

In [9]:
params_rf = best_parameters(exp_test_rf)
params_svm = best_parameters(exp_test_svm)

print(json.dumps(params_rf, indent=4))
print(json.dumps(params_svm, indent=4))

[
    [
        0.596445251683727,
        {
            "min_samples_split": 0.04466339369901853,
            "max_features": 0.0010292429905777445,
            "num_arvores": 1
        }
    ],
    [
        0.5954073820857066,
        {
            "min_samples_split": 0.08842136011502941,
            "max_features": 0.22848035048457271,
            "num_arvores": 1
        }
    ],
    [
        0.5923505029598322,
        {
            "min_samples_split": 0.10251976896112781,
            "max_features": 0.4785340513058907,
            "num_arvores": 1
        }
    ],
    [
        0.5886801037590487,
        {
            "min_samples_split": 0.12650146532653397,
            "max_features": 0.32474851683945516,
            "num_arvores": 2
        }
    ],
    [
        0.5874204938135473,
        {
            "min_samples_split": 0.1164735770300016,
            "max_features": 0.45935262471729577,
            "num_arvores": 1
        }
    ]
]
[
    [
        0.784123350272648

## Verificação dos parâmetos

In [10]:
from competicao_am.gerar_resultado_teste import EXP_COST, GAMMA, MIN_SAMPLES_SPLIT, MAX_FEATURES, NUM_ARVORES

correct_params = {
    'exp_cost': EXP_COST in [p[1]['exp_cost'] for p in params_svm],
    'gamma': GAMMA in [p[1]['gamma'] for p in params_svm],

    'min_samples_split': MIN_SAMPLES_SPLIT in [p[1]['min_samples_split'] for p in params_rf],
    'max_features': MAX_FEATURES in [p[1]['max_features'] for p in params_rf],
    'num_arvores': NUM_ARVORES in [p[1]['num_arvores'] for p in params_rf]
}

for k, v in correct_params.items():
    if not v:
        print(f'Param "{k}" might be outdated!')

## Teste do método usado na ausência de um resumo

In [11]:
rf_classifier = RandomForestClassifier(
        min_samples_split=MIN_SAMPLES_SPLIT, max_features=MAX_FEATURES, n_estimators=NUM_ARVORES, random_state=2)

df_rf_treino = df_treino[~df_treino.resumo.isna()]
df_rf_predict = df_treino[df_treino.resumo.isna()]

mc_rf = mc.MetodoCompeticaoValidacaoRF(rf_classifier)
results_rf = mc_rf.eval(df_rf_treino, df_rf_predict, 'genero')

mc_always_comedy = mc.MetodoCompeticaoValidacaoComedy(rf_classifier)
results_always_comedy = mc_always_comedy.eval(df_rf_treino, df_rf_predict, 'genero')

In [12]:
print(f'results_rf: {results_rf.macro_f1}')
print(f'results_always_comedy: {results_always_comedy.macro_f1}')

results_rf: 0.5554760200429492
results_always_comedy: 0.47727272727272724


## Gerar predict

In [13]:
NUM_GRUPO = 11

def gerar_predict(version: int):
    """
    Função que gera o predict evitando que ele seja gerado cada vez que o notebook é executado
    """
    predict_info = None
    PREDICT_INFO_PATH = 'predict_info.json'
    
    try:
        with open(PREDICT_INFO_PATH, 'r') as predict_info_file:
            predict_info = json.load(predict_info_file)
    except:
        predict_info = {}
    
    if 'version' in predict_info and predict_info['version'] == version:
        print('Predict already generated')
        return
    
    predict_info = {'version': version}
    gerar_saida_teste(df_predict_final, 'genero', NUM_GRUPO)

    with open(PREDICT_INFO_PATH, 'w') as predict_info_file:
        json.dump(predict_info, predict_info_file)

In [14]:
gerar_predict(1)

Predict already generated
